In [2]:
%matplotlib agg
%matplotlib agg

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import scipy.stats as st
import importlib
import os
import sys
import platform
import qgrid
import datetime
from scipy.interpolate import make_interp_spline, BSpline
import pickle
import random

from numba import njit

from sync_class import SyncPhenosys
from eda_class import SpikesEDA
from behavior_class import BehaviorAnalysis
from sda_class import SpikesSDA
from report_class import SpikesReport


%load_ext autoreload
%autoreload 2

#pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 100)


#plt.style.use('ggplot')
#plt.style.use('gadfly.mplstyle')

# %config InlineBackend.figure_format = 'retina'
#plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")


#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')


# ============================================================================================================================
# specify file path & sessions

windows_folder = r"C:/Users/User/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"
linux_folder = "/home/max/ExpanDrive/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"
mac_folder = "/Users/max/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"

# list -> [ session_name, [ttl_missing_rows], load_spikes, [spikes_trials_skip] ]
se_li = [('JG14_190621', [1900,1931,1996,2058,2127],True,[(0,6),(215,'end')]),
         ('JG14_190619', [111, 2781],False,[]),
         ('JG14_190626', [1428, 1824, 1838, 2861, 2910, 3089, 3245, 3430, 3443],False,[]),
         ('JG15_190722', [2094, 2574, 2637, 2808, 2831, 3499],False,[]),
         ('JG15_190725', [366,711,1487,1578,1659,2221,2666,2720,2769,2847,3371,3476],False,[]),
         ('JG18a_190814', [405,2621,2693,2770,2959,3015,3029,3038,3048],False,[]),
         ('JG18b_190828', [1744, 2363, 2648, 2701, 2731, 2778,2953,2967],True,[]),
         ]



# ============================================================================================================================
# Load sessions functions

def add_session(session, gamble_side, sync_obj, missing_rows_ttl, combined, trials, behavior, spikes, trials_skip):
    session_dict = {
        'session':session,
        'gamble_side':gamble_side,
        #'sync_obj':sync_obj,
        'missing_rows_ttl':missing_rows_ttl,
        'combined':combined,
        'trials':trials,
        'behavior':behavior,
        'trials_skip':trials_skip,
    }
    if spikes != False:
        session_dict['spikes']=spikes
        
    # add session dict to all sessions dict
    all_sessions_dict[session]=session_dict
    return session_dict


def load_session(session, missing_rows_ttl=[], lo_spikes=False, trials_skip=[]):

    # load calss and set folder depending on platform

    # load calss and set folder depending on platform
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder + '/' + session
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder + r"/" + session
    elif platform.system() == 'Darwin':
        folder = mac_folder + r"/" + session

    sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl)   
    gamble_side = sync_obj.gamble_side

    # first find missing rows
    #missing_rows_csv = []
    #combined = sync_obj.combine_dataframes(missing_rows_ttl, missing_rows_csv, 'channel 1', sync_obj.csv, False)
    #trials_df = sync_obj.get_trials(combined)
    behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
    
    if lo_spikes:
        spikes_obj = SpikeAnalysis(behavior_obj)
    #    spikes = SpikeAnalysis(session, folder, sync_obj)
    #    for a,b in trials_skip:
    #        if b == 'end':
    #            spikes.trials_df.loc[a:,'select'] = False
    #        else:
    #            spikes.trials_df.loc[a:b,'select'] = False
    #else:
    #    spikes = False

    return session, gamble_side, sync_obj, behavior_obj, spikes_obj


def load_all_sessions(se_li):
    all_sessions = dict()
    for session, missing_rows_ttl, lo_spikes, trials_skip in se_li:
        session, gamble_side, sync_obj, combined, trials, behavior, spikes = load_session(session, missing_rows_ttl, lo_spikes, trials_skip)
        
        session_dict = add_session(session, gamble_side, sync_obj, missing_rows_ttl, combined, trials, behavior, spikes, trials_skip)  
        all_sessions[session]=session_dict

    return all_sessions


        
# ============================================================================================================================
# save & load sessions from pickl functions

# save dict to pickl
def save_all_sessions_dict_pickl():
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder
    elif platform.system() == 'Darwin':
        folder = mac_folder 
        
    file = folder + '/all_sessions_dict.pkl'
    with open(file, 'wb') as dump:
        pickle.dump(all_sessions_dict, dump)

# load dict from pickl
def load_all_sessions_dict_pickl():
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder
    elif platform.system() == 'Darwin':
        folder = mac_folder 
        
    file = folder + '/all_sessions_dict.pkl'
    
    with open(file, 'rb') as dump:
        all_sessions_dict = pickle.load(dump)
    return all_sessions_dict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pipeline Example

In [ ]:
session = 'NAME'
folder = mac_folder + r"/" + session
# sync
rows_missing_ttl = []
# behavior
deselect_trials = [(X,X),(X,'end')]
# sda
window = 2000
iterations = 1000
bins = 50

#sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 
#behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
#eda_obj = SpikesEDA(behavior_obj)
#sda_obj = SpikesSDA(eda_obj)
#report_obj = SpikesReport(eda_obj)

"""
NAME = type('obj', (object,), 
                   {'sync':sync_obj,
                    'behavior':behavior_obj,
                    'eda':eda_obj,
                    'sda': sda_obj,
                    'report': report_obj,
                   })
"""

In [ ]:
%%time
%matplotlib agg
NAME.eda.generate_plots()

In [ ]:
%%time
%matplotlib agg
NAME.sda.generate_plots(window,iterations,bins,individual=True,load=True)

In [8]:
NAME.sda.info_df

,session,tot. clusters,nr. good,nr. mua,nr. noise,tot. trials,good trials,selected trials,rw block 1,rw block 2,...,3 rw_safe all,3 rw_safe all neurons,3 norw_safe before,3 norw_safe before neurons,3 norw_safe across,3 norw_safe across neurons,3 norw_safe after,3 norw_safe after neurons,3 norw_safe all,3 norw_safe all neurons
1,JG14_190621,167,92,61,14,570,267,208,0.75,0.25,...,10,"[16, 18, 19, 21, 36, 38, 39, 41, 45, 71]",1,[16],0,[],0,[],1,[16]


In [ ]:
%%time
NAME.report.generate_report()

In [11]:
del JG14_190621

**Sync**

**1. define session & folder -> must be name of folder in parent folder with all behavior data**
    - folder/session/electrophysiology/digitalin.dat
    
**2. prepare csv from Phenosys**
    - partent/session/behavior/output.csv
    - example included in repository
    - headers: DateTime,SystemMsg,probability,gambl side
    - all other columns must be deleted

**3. create sync object** 
    - sync_obj = Sync(session, folder, number_of_ttl_channels_in_ditialin.dat, channel_with_information(default=1))

**4. investigate ttl signal and how it is overlapping with csv output (1.3)**

**5. create combined dataframe & separated trials**



In [ ]:
# load session initially
sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 

In [ ]:
ttl1 = sync_obj.ttl_signals['channel 1']
qgrid.show_grid(ttl1)

In [ ]:
csv = sync_obj.csv
qgrid.show_grid(csv)

In [ ]:
# first find missing rows
missing_rows = ()
combined = sync_obj.combine_dataframes(missing_rows, additonal_rows, 'channel 1', csv)

qgrid.show_grid(combined)

---

**Overview**

In [ ]:
sync_obj.ttl_info_channel

In [ ]:
combined['Delta (TTL-CSV)'].plot()

In [ ]:
combined.iloc[1720:1730]

---

**Behavior**

**1. get selected_trials**\
plot response time and no response behavior \
subselect trials in between

In [ ]:
NAME.eda.plt_trial_length()

In [ ]:
#spikes.trials_df.loc[np.r_[0:6, 215:], 'select']
# start
NAME.eda.trials_df.loc[0:1, 'select'] = False
# end
NAME.eda.trials_df.loc[215:, 'select'] = False
NAME.eda.trials_df

In [ ]:
NAME.eda.plt_trial_hist_and_fit(
    NAME.eda.trials_df.loc[NAME.eda.trials_df.loc[:, 'select'], 'length'])

In [ ]:
deselect_trials = [(X,X),(X,'end')]

---

**EDA**

---

**SDA**

---

# JG14_190621

In [18]:
%%time

session = 'JG14_190621'
folder = mac_folder + r"/" + session
# sync
rows_missing_ttl = [1900,1931,1996,2058,2127]
# behavior
deselect_trials = [(0,6),(215,'end')]
# sda
window = 2000
iterations = 1000
bins = 50

sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 
behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
eda_obj = SpikesEDA(behavior_obj)
sda_obj = SpikesSDA(eda_obj)
report_obj = SpikesReport(eda_obj)

JG14_190621 = type('obj', (object,), 
                   {'sync':sync_obj,
                    'behavior':behavior_obj,
                    'eda':eda_obj,
                    'sda': sda_obj,
                    'report': report_obj,
                   })


CPU times: user 53.3 s, sys: 9.32 s, total: 1min 2s
Wall time: 1min 3s


In [ ]:
%%time
%matplotlib agg
JG14_190621.eda.generate_plots()

In [ ]:
%%time
%matplotlib agg
JG14_190621.sda.generate_plots(window,iterations,bins,individual=True,load=True)

In [8]:
JG14_190621.sda.info_df

,session,tot. clusters,nr. good,nr. mua,nr. noise,tot. trials,good trials,selected trials,rw block 1,rw block 2,...,3 rw_safe all,3 rw_safe all neurons,3 norw_safe before,3 norw_safe before neurons,3 norw_safe across,3 norw_safe across neurons,3 norw_safe after,3 norw_safe after neurons,3 norw_safe all,3 norw_safe all neurons
1,JG14_190621,167,92,61,14,570,267,208,0.75,0.25,...,10,"[16, 18, 19, 21, 36, 38, 39, 41, 45, 71]",1,[16],0,[],0,[],1,[16]


In [ ]:
%%time
JG14_190621.report.generate_report()

In [11]:
del JG14_190621

# JG18b_190828

In [3]:
%%time

session = 'JG18b_190828'
folder = mac_folder + r"/" + session
# sync
rows_missing_ttl = [1744, 2363, 2648, 2701, 2731, 2778,2953,2967]
# behavior
deselect_trials = [(0,0),(204,'end')]
# eda
skip_clusters = [116, 280]
# sda
window = 2000
iterations = 1000
bins = 50

sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 
behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
eda_obj = SpikesEDA(behavior_obj)
sda_obj = SpikesSDA(eda_obj)
report_obj = SpikesReport(eda_obj)

JG18b_190828 = type('obj', (object,), 
                   {'sync':sync_obj,
                    'behavior':behavior_obj,
                    'eda':eda_obj,
                    'sda': sda_obj,
                    'report': report_obj,
                   })



CPU times: user 1min 21s, sys: 1.77 s, total: 1min 23s
Wall time: 1min 23s


In [4]:
%%time
%matplotlib agg
JG18b_190828.eda.generate_plots()

[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[

[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[

[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[(0.25, 61), (0.125, 161)]
[

ValueError: too many values to unpack (expected 2)

In [7]:
%%time
%matplotlib agg
JG18b_190828.sda.generate_plots(window,iterations,bins,individual=False,load=False)

all mean vs 2sigma -> significant neurons
rw mean vs 2sigma -> significant neurons
norw mean vs 2sigma -> significant neurons
gamble mean vs 2sigma -> significant neurons
safe mean vs 2sigma -> significant neurons
rw_gamble mean vs 2sigma -> significant neurons
norw_gamble mean vs 2sigma -> significant neurons
rw_safe mean vs 2sigma -> significant neurons
norw_safe mean vs 2sigma -> significant neurons
block1_all mean vs 2sigma -> significant neurons
block1_rw mean vs 2sigma -> significant neurons
block1_norw mean vs 2sigma -> significant neurons
block1_gamble mean vs 2sigma -> significant neurons
block1_rw_gamble mean vs 2sigma -> significant neurons
block1_norw_gamble mean vs 2sigma -> significant neurons
block1_safe mean vs 2sigma -> significant neurons
block1_rw_safe mean vs 2sigma -> significant neurons
block1_norw_safe mean vs 2sigma -> significant neurons
block2_all mean vs 2sigma -> significant neurons
block2_rw mean vs 2sigma -> significant neurons
block2_norw mean vs 2sigma -

In [8]:
JG18b_190828.sda.info_df

,session,tot. clusters,nr. good,nr. mua,nr. noise,tot. trials,good trials,selected trials,rw block 1,rw block 2,...,3 rw_safe all,3 rw_safe all neurons,3 norw_safe before,3 norw_safe before neurons,3 norw_safe across,3 norw_safe across neurons,3 norw_safe after,3 norw_safe after neurons,3 norw_safe all,3 norw_safe all neurons
1,JG18b_190828,385,187,174,24,727,375,203,0.75,0.25,...,18,"[48, 52, 53, 60, 61, 64, 66, 67, 86, 96, 97, 1...",0,[],0,[],0,[],0,[]


In [ ]:
%%time
#%matplotlib agg
JG18b_190828.report.generate_report()

In [9]:
JG18b_190828.eda.gamble_side

'right'